In [ ]:
import pandas as pd
import numpy as np
from tableone import TableOne
from scipy import stats
from sklearn import metrics
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

# tableone

In [ ]:
result=pd.read_csv('POAF_result.csv')
result_fortable1=result.copy()

In [ ]:
agegroup=[]
for i in range(len(result_fortable1)):
    if result_fortable1['age'][i]<60:
        agegroup.append(1)
    elif result_fortable1['age'][i]<70:
        agegroup.append(2)
    elif result_fortable1['age'][i]<80:
        agegroup.append(3)
    elif result_fortable1['age'][i]>=80:
        agegroup.append(4)
    else:
        print('wrong')
result_fortable1['agegroup']=agegroup

EFbelow30=[]
for i in range(len(result_fortable1)):
    if result_fortable1['EF'][i]<30:
        EFbelow30.append(1)
    else:
        EFbelow30.append(0)
result_fortable1['EFbelow30']=EFbelow30

eGFRbelow15=[]
for i in range(len(result_fortable1)):
    if result_fortable1['eGFR'][i]<15:
        eGFRbelow15.append(1)
    else:
        eGFRbelow15.append(0)
result_fortable1['eGFRbelow15']=eGFRbelow15

In [ ]:
def table1(catcolumn,catcolumns,dfdfdf2):
    nonnormallist=[]
    def _normality(self, x):
        #print(x.name)

        if len(x.values[~np.isnan(x.values)]) >= 20:
            p = stats.shapiro(x.values).pvalue
        else:
            p = None
        # dropna=False argument in pivot_table does not function as expected
        # return -1 instead of None
        if pd.isnull(p):
            return -1
        if p<=0.05:
            nonnormallist.append(x.name)
        return p

    TableOne._normality=_normality

    def my_custom_test(group1, group2):
        """
        Hypothesis test for test_self_defined_statistical_tests
        """
        my_custom_test.__name__ = "mannwhitneyu"
        _, pval= scipy.stats.mannwhitneyu(group1, group2)
        return pval

    nonnormallist=[]
    def _normality(self, x):
        #print(x.name)

        if len(x.values[~np.isnan(x.values)]) >= 20:
            p = stats.shapiro(x.values).pvalue
        else:
            p = None
        # dropna=False argument in pivot_table does not function as expected
        # return -1 instead of None
        if pd.isnull(p):
            return -1
        if p<=0.05:
            nonnormallist.append(x.name)
        return p

    TableOne._normality=_normality

    table1=TableOne(dfdfdf2,categorical=catcolumns,groupby=[catcolumn],normal_test=True,pval=True,htest_name=True,decimals=3)
    nonnormallist=list(set(nonnormallist))
    nonnormallist

    table1=TableOne(dfdfdf2,categorical=catcolumns,groupby=[catcolumn],normal_test=True,pval=True,htest_name=True,nonnormal=nonnormallist,decimals=3)
    try:
        os.mkdir(newtablename)
    except:
        pass
    try:
        os.mkdir(newtablename+'/table1')
    except:
        pass
    catcolumn=catcolumn.replace(' ','_')
    catcolumn=catcolumn.replace('/','_')
    table1.to_html('figures/temptable1_'+catcolumn+'.html')

In [ ]:
categor=['valve_surgery',
         'sex',
         'emergency',
         'HTN',
         'CKD',
         'old CVA',
         'DM',
         'A.fib',
         'liver cirrhosis',
         'CHF',
         'old MI',
         'COPD',
         'acuteMI (1WK)',
         'recentMI (3mon)',
         'BB',
         'CCB',
         'ACEi',
         'ARB',
         'Statin',
         'PO Nitrate',
         'diuretics',
         'Warfarin',
         'heparinization',
         'IV nitrate',
         'NOAC',
         'ECMOVADIABP_label',
         'new AF','agegroup','EFbelow30','eGFRbelow15']

contin2=['age',
         'height',
         'weight',
         'BSA',
         'SBP',
         'DBP',
         'WBC',
         'Hb',
         'Plt (*1000)',
         'Na',
         'K',
         'BUN',
         'Cr',
         'eGFR',
         'OT',
         'PT',
         'Albumin',
         'Glu',
         'PT_sec',
         'PTT',
         'EF',
         'prediction_score','POAF score']

totalcolumns=categor+contin2

In [ ]:
result_table1=result_fortable1[totalcolumns]

In [ ]:
table1('new AF',categor,result_table1)

# ROC PR curves, performance

In [ ]:
POAFresult=pd.read_csv('POAF_result.csv')

In [ ]:
POAFresult=pd.read_csv('POAF_result.csv')
resulttemp=POAFresult.copy()

resulttemp['opdate'] = pd.to_datetime(resulttemp['opdate'], format='%Y-%m-%d')
resulttemp1 = resulttemp[resulttemp['opdate'] <= '2021-12-31']
result = resulttemp[resulttemp['opdate'] >= '2022-01-01']

print(f"Rows in df1: {len(resulttemp1)}")
print(f"Rows in df2: {len(result)}")
resulttemp1.reset_index(inplace=True,drop=True)
result.reset_index(inplace=True,drop=True)

NEWscore=list(pd.read_csv('NEW_POAF_Score_predicted_probs.csv')['x'])
result['NEWscore']=NEWscore

POAF_score_list=[]
for j in range(len(result)):
    POAF_score=0
    if result['age'][j]>=80:
        POAF_score+=3
    elif result['age'][j]>=70:
        POAF_score+=2
    elif result['age'][j]>=60:
        POAF_score+=1
    else:
        POAF_score+=0

    if result['COPD'][j]==1:
        POAF_score+=1

    if result['eGFR'][j]<15:
        POAF_score+=1

    if result['emergency'][j]==1:
        POAF_score+=1
        
    if result['ECMOVADIABP_label'][j]==1:
        POAF_score+=1

    if result['EF'][j]<=30:
        POAF_score+=1

    if result['valve_surgery'][j]==1:
        POAF_score+=1
        
    POAF_score_list.append(POAF_score)
    
result['POAF score']=POAF_score_list
    
fpr_temp,tpr_temp,threshold=metrics.roc_curve(result['new AF'],POAF_score_list,pos_label=1)
AUROC=metrics.auc(fpr_temp,tpr_temp)    
print('@@@@ POAF score AUROC',AUROC)

fpr_temp,tpr_temp,threshold=metrics.roc_curve(result['new AF'],result['prediction_score'],pos_label=1)
AUROC=metrics.auc(fpr_temp,tpr_temp)    
print('@@@@ AI score AUROC',AUROC)

for i in [4]:
    result['POAF_AI score']=result['POAF score']+result['prediction_score']*i

    fpr_temp,tpr_temp,threshold=metrics.roc_curve(result['new AF'],result['POAF_AI score'],pos_label=1)
    AUROC=metrics.auc(fpr_temp,tpr_temp)    
    print(i,'@@@@ POAF_AI score AUROC',AUROC)

fpr_temp,tpr_temp,threshold=metrics.roc_curve(result['new AF'],result['NEWscore'],pos_label=1)
AUROC=metrics.auc(fpr_temp,tpr_temp)    
print('@@@@ NEWscore AUROC',AUROC)

In [ ]:
def ROCcurve(df):
    plt.figure(figsize=(10,10))
    plt.xlim([-0.00, 1.00])
    plt.ylim([-0.00, 1.00])
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='grey',alpha=.8)

    fpr_POAF,tpr_POAF,threshold=metrics.roc_curve(df['new AF'],df['POAF score'],pos_label=1)
    fpr_POAF_AI,tpr_POAF_AI,threshold=metrics.roc_curve(df['new AF'],df['POAF_AI score'],pos_label=1)
    fpr_AI,tpr_AI,threshold=metrics.roc_curve(df['new AF'],df['prediction_score'],pos_label=1)
    fpr_newscore,tpr_newscore,threshold=metrics.roc_curve(df['new AF'],df['NEWscore'],pos_label=1)

    AUROC_POAF=metrics.auc(fpr_POAF,tpr_POAF)
    AUROC_POAF_AI=metrics.auc(fpr_POAF_AI,tpr_POAF_AI)
    AUROC_AI=metrics.auc(fpr_AI,tpr_AI)
    AUROC_newscore=metrics.auc(fpr_newscore,tpr_newscore)



    plt.plot(fpr_POAF, tpr_POAF, color='red',label='POAF-Score AUROC %0.3f' % (AUROC_POAF) ,lw=1, alpha=1)
    plt.plot(fpr_AI, tpr_AI, color='green',label=r'AI-ECG-AF score AUROC %0.3f' % (AUROC_AI) ,lw=1, alpha=1)
    plt.plot(fpr_POAF_AI, tpr_POAF_AI, color='purple',label=r'POAF-AI-Score AUROC %0.3f' % (AUROC_POAF_AI) ,lw=1, alpha=1)
    plt.plot(fpr_newscore, tpr_newscore, color='blue',label=r'New-POAF-Score AUROC %0.3f' % (AUROC_newscore) ,lw=1, alpha=1)
    plt.xlabel('1-Specificity', fontsize=17)
    plt.ylabel('Sensitivity', fontsize=17)
    plt.title('ROC curves', fontsize=19)
    #plt.legend(loc="lower right", fontsize=29)
    plt.xticks(np.arange(0.2, 1.2, step=0.2))
    plt.yticks(np.arange(0, 1.2, step=0.2))
    plt.xticks(fontsize =15)
    plt.yticks(fontsize =15)
    leg=plt.legend(loc="lower right", fontsize=17)
    for legobj in leg.legendHandles:
        legobj.set_linewidth(2.0)
    #plt.show()

    plt.savefig('figures/ROC.png',transparent=True)
    plt.close()

In [ ]:
ROCcurve(result)

In [ ]:
def PRcurve(df):
    plt.figure(figsize=(10,10))
    plt.xlim([-0.00, 1.00])
    plt.ylim([-0.00, 1.00])

    precision_POAF, recall_POAF, _ = metrics.precision_recall_curve(df['new AF'], df['POAF score'], pos_label=1)
    precision_POAF_AI, recall_POAF_AI, _ = metrics.precision_recall_curve(df['new AF'], df['POAF_AI score'], pos_label=1)
    precision_AI, recall_AI, _ = metrics.precision_recall_curve(df['new AF'], df['prediction_score'], pos_label=1)
    precision_newscore, recall_newscore, _ = metrics.precision_recall_curve(df['new AF'], df['NEWscore'], pos_label=1)

    AP_POAF = metrics.average_precision_score(df['new AF'], df['POAF score'])
    AP_POAF_AI = metrics.average_precision_score(df['new AF'], df['POAF_AI score'])
    AP_AI = metrics.average_precision_score(df['new AF'], df['prediction_score'])
    AP_newscore = metrics.average_precision_score(df['new AF'], df['NEWscore'])

    plt.plot(recall_POAF, precision_POAF, color='red', label='POAF-Score AUPRC %0.3f' % (AP_POAF), lw=1, alpha=1)
    plt.plot(recall_AI, precision_AI, color='green', label=r'AI-ECG-AF score AUPRC %0.3f' % (AP_AI), lw=1, alpha=1)
    plt.plot(recall_POAF_AI, precision_POAF_AI, color='purple', label=r'POAF-AI-Score AUPRC %0.3f' % (AP_POAF_AI), lw=1, alpha=1)
    plt.plot(recall_newscore, precision_newscore, color='blue', label=r'New-POAF-Score AUPRC %0.3f' % (AP_newscore), lw=1, alpha=1)

    plt.xlabel('Recall', fontsize=17)
    plt.ylabel('Precision', fontsize=17)
    plt.title('PR curves', fontsize=19)
    plt.xticks(np.arange(0.2, 1.2, step=0.2))
    plt.yticks(np.arange(0, 1.2, step=0.2))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    leg = plt.legend(loc="lower left", fontsize=17)
    for legobj in leg.legendHandles:
        legobj.set_linewidth(2.0)
    plt.savefig('figures/PR.png',transparent=True)
    plt.close()

In [ ]:
PRcurve(result)

In [ ]:
POAF_score_list=[]
for j in range(len(resulttemp1)):
    POAF_score=0
    if resulttemp1['age'][j]>=80:
        POAF_score+=3
    elif resulttemp1['age'][j]>=70:
        POAF_score+=2
    elif resulttemp1['age'][j]>=60:
        POAF_score+=1
    else:
        POAF_score+=0

    if resulttemp1['COPD'][j]==1:
        POAF_score+=1

    if resulttemp1['eGFR'][j]<15:
        POAF_score+=1

    if resulttemp1['emergency'][j]==1:
        POAF_score+=1
        
    if resulttemp1['ECMOVADIABP_label'][j]==1:
        POAF_score+=1

    if resulttemp1['EF'][j]<=30:
        POAF_score+=1

    if resulttemp1['valve_surgery'][j]==1:
        POAF_score+=1
        
    POAF_score_list.append(POAF_score)
    
resulttemp1['POAF score']=POAF_score_list
i=4
resulttemp1['POAF_AI score']=resulttemp1['POAF score']+resulttemp1['prediction_score']*i
NEWscore=list(pd.read_csv('NEW_POAF_Score_predicted_probs2.csv')['x'])
resulttemp1['NEWscore']=NEWscore

In [ ]:
def calculate_metrics(y_true, y_scores, y_true2, y_scores2):
    # Calculate ROC curve
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_scores, pos_label=1)
    # Calculate Precision-Recall curve
    precision, recall, _ = metrics.precision_recall_curve(y_true, y_scores, pos_label=1)

    # Calculate Youden's J index
    youden_j = tpr - fpr
    max_j_idx = np.argmax(youden_j)
    best_threshold = thresholds[max_j_idx]

    # Calculate metrics at the best threshold
    y_pred2 = (y_scores2 >= best_threshold).astype(int)
    accuracy = metrics.accuracy_score(y_true2, y_pred2)
    sensitivity = metrics.recall_score(y_true2, y_pred2)  # Same as TPR
    specificity = metrics.recall_score(y_true2, y_pred2, pos_label=0)  # TNR
    ppv = metrics.precision_score(y_true2, y_pred2)
    npv = metrics.precision_score(y_true2, y_pred2, pos_label=0)
    f1 = metrics.f1_score(y_true2, y_pred2)

    # Calculate AUROC and AP
    fpr, tpr, thresholds = metrics.roc_curve(y_true2, y_scores2, pos_label=1)
    auroc = metrics.auc(fpr, tpr)
    ap = metrics.average_precision_score(y_true2, y_scores2)

    return {
        'Accuracy': accuracy,
        'Sensitivity (TPR)': sensitivity,
        'Specificity (TNR)': specificity,
        'PPV': ppv,
        'NPV': npv,
        'F1 Score': f1,
        'AUROC': auroc,
        'AP': ap
    }

# Calculate metrics for each score
metrics_POAF = calculate_metrics(resulttemp1['new AF'], resulttemp1['POAF score'],result['new AF'], result['POAF score'])
metrics_POAF_AI = calculate_metrics(resulttemp1['new AF'], resulttemp1['POAF_AI score'],result['new AF'], result['POAF_AI score'])
metrics_AI = calculate_metrics(resulttemp1['new AF'], resulttemp1['prediction_score'],result['new AF'], result['prediction_score'])
metrics_newscore = calculate_metrics(resulttemp1['new AF'], resulttemp1['NEWscore'],result['new AF'], result['NEWscore'])

# Combine all metrics into a DataFrame
metrics_df = pd.DataFrame({
    'POAF': metrics_POAF,
    'AI-ECG-AF': metrics_AI,
    'POAF_AI': metrics_POAF_AI,
    'New Score': metrics_newscore
})

# Transpose the DataFrame for better readability
metrics_df = metrics_df.transpose()

In [ ]:
metrics_df